<a href="https://colab.research.google.com/github/Pt-home/Notebooks/blob/main/Nomades2_0_03_%D0%A6%D0%B5%D0%BF%D0%B8_%D0%9C%D0%B0%D1%80%D0%BA%D0%BE%D0%B2%D0%B0_%D0%BD%D0%B0_nltk_%D0%B1%D0%B5%D0%BB%D1%8B%D0%B5_%D1%81%D1%82%D0%B8%D1%85%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install numpy
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
f = 'sample_data/brodsky_poetry.txt'

In [4]:
import re

# Load the text data
with open(f, "r", encoding='utf8') as file:
    text = file.read()
    
text = text.upper()

while text.find('  ') != -1:
  text = re.sub(r'  ', ' ', text)
while text.find('--') != -1:
  text = re.sub(r'--', '-', text)

text = re.sub(r'\s-\s', ' — ', text)
text = re.sub(r'[^-А-ЯЁ—\s]', '', text)

In [5]:
#Подсчет пробелов
def count_syllables(word):
    answer = 0
    for el in "УЕЫАОЭЯИЮЁ":
        answer += word.count(el)
    return answer

In [6]:
import nltk
nltk.download('punkt')

import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


BIGRAMMS

In [7]:
# Define the Markov model
premodel = nltk.probability.ConditionalFreqDist()

words = nltk.word_tokenize(text)

for i in range(len(words)-1):
    premodel[words[i]][words[i+1]] += 1

In [8]:
model = nltk.ConditionalProbDist(premodel, nltk.MLEProbDist)
len(model)


38288

In [10]:
def generate_poem(model, num_lines, num_syllables, start_condition):
     
    # Создаем список для хранения сгенерированных строк
    lines = []

    # Создаем начальное условие 
    condition = start_condition
    
    # Генерируем каждую строку
    for i in range(num_lines):

        if i == 0: #If this start line
            line_words = [condition]
            line_syllables = count_syllables(condition)
        else:
            line_words = []
            line_syllables = 0

        # Генерируем строку, пока количество слогов не достигнет заданного значения
        while line_syllables < num_syllables:

            # Получаем вероятностное распределение для следующего слова и генерируем его
            word = model[condition].generate()

            # Считаем количество слогов в слове
            syllable_count = count_syllables(word)
            
            # Если количество слогов превышает оставшееся количество для строки, то выбираем новое слово КОСТЫЛЬ
            if line_syllables + syllable_count > num_syllables:
                condition = random.choice(list(model.conditions()))
                continue

            # Добавляем слово в строку и увеличиваем количество слогов
            line_words.append(word)
            line_syllables += syllable_count
                        
            # Обновляем текущее условие для следующей генерации
            condition = word

       # Добавляем строку в список
        lines.append(' '.join(line_words))
        
    # Возвращаем сгенерированные строки
    return lines

In [11]:
num_lines = 4
num_syllables = 16

# Создаем начальное условие из случайных слов в модели
start_condition = random.choice(list(model.conditions()))

#Генерируем
poem = generate_poem(model, num_lines, num_syllables, start_condition)

#Выводим
for line in poem:
    print(line)

КЛИНОПИСЬ МЫСЛЕЙ БЕШЕНЫХ ВОПРОСОВ КАК СТРАННО ТЕБЯ
И Я ГОТОВ ОТТОГО И СЕРПАХ Я ЧЕМ МУСКУЛИСТЕЙ
КОРНИ ВЦЕПЛЯЮТСЯ В НЕБО БУДЕТ МОГУЧ И ВЕСЛАМИ
ПО ПОДОЛУ БАБЫ ПРЯЧУТ ПРЕКРАСНЫХ ВИДЕНИЙ ЖИВОЙ


In [12]:
num_lines = 4
num_syllables = 16

# Создаем начальное условие из случайных слов в модели
start_condition = "НЫНЧЕ"

#Генерируем
poem = generate_poem(model, num_lines, num_syllables, start_condition)

#Выводим
for line in poem:
    print(line)

НЫНЧЕ МНЕ ЛЬ ОДНО ИЗ ФАНЕРЫ В ИНДИЮ ТИГРЫ ИЛИ
ЖИВ ТО СТАРИТ ТО ЧТО ЭХА А МУЗЫКА ГРЕМЕЛА НА
МЕСТО СТАВИТ МИКЕЛИНУ РАКОМ ЭХ ТОПОРИК В ЛЕСУ
НЕТ ЗЕМНОЙ ЯНВАРЯ — ЦВЕТКОМ В ЗЕРКАЛЕ ТВОРЯЩЕЕСЯ


TRIGRAMS

In [13]:
# Define the Markov model
premodel = nltk.probability.ConditionalFreqDist()

words = nltk.word_tokenize(text)

for i in range(len(words)-2):
    key = words[i] + " " + words[i+1]
    premodel[key][words[i+2]] += 1

In [14]:
model = nltk.ConditionalProbDist(premodel, nltk.MLEProbDist)
len(model)

129684

In [15]:
def generate_poem_tri(model, num_lines, num_syllables, start_condition):
     
    # Создаем список для хранения сгенерированных строк
    lines = []

    # Создаем начальное условие 
    condition = start_condition

    # Создаем список всех слов
    all_words = condition.split()
    
    # Генерируем каждую строку
    for i in range(num_lines):

        if i == 0: #If this start line
            line_words = condition.split()
            line_syllables = count_syllables(line_words[-2]) + count_syllables(line_words[-1]) 
        else:
            line_words = []
            line_syllables = 0

        # Генерируем строку, пока количество слогов не достигнет заданного значения
        while line_syllables < num_syllables:

            # Получаем вероятностное распределение для следующего слова и генерируем его
            word = model[condition].generate()

            # Считаем количество слогов в слове
            syllable_count = count_syllables(word)
            
            # Если количество слогов превышает оставшееся количество для строки, то выбираем новое слово КОСТЫЛЬ
            if line_syllables + syllable_count > num_syllables:
                condition = random.choice(list(model.conditions()))
                all_words = all_words[:-2] + condition.split()
                continue

            # Добавляем слово в строку и увеличиваем количество слогов
            line_words.append(word)
            all_words.append(word)
            line_syllables += syllable_count
                        
            # Обновляем текущее условие для следующей генерации
            condition = all_words[-2] + " " + all_words[-1]

       # Добавляем строку в список
        lines.append(' '.join(line_words))
        
    # Возвращаем сгенерированные строки
    return lines

In [19]:
num_lines = 4
num_syllables = 16

# Создаем начальное условие из случайных слов в модели
start_condition = random.choice(list(model.conditions()))

#Генерируем
poem = generate_poem_tri(model, num_lines, num_syllables, start_condition)

#Выводим
for line in poem:
    print(line)

ГЛАЗАМ СМОТРЕТЬ НЕВМОЧЬ КАК КРАЙ СТЕКЛА ЗАЛИТЫЙ СВЕТОМ ВЗОР
ЗАЛИТЫЙ ТЬМОЙ КАДЫК ЗАПЕРТЫЙ ОТ ГОСТЕЙ С ВЕЧНЫМ ПРОФИЛЬ
БЛЕДНЕЙ ГРЕХА ЗАЛИВАЯ ГЛАЗА НА СЕКУНДУ ЗВУКА
ТАК ДОВОЛЬНО СПИЧКИ ЧТОБЫ РАЗЖЕЧЬ ПЛИТУ ТАК СТЕННЫЕ


In [21]:
num_lines = 4
num_syllables = 16

# Создаем начальное условие из случайных слов в модели
start_condition = "НЫНЧЕ ВЕТРЕНО"

#Генерируем
poem = generate_poem_tri(model, num_lines, num_syllables, start_condition)

#Выводим
for line in poem:
    print(line)

НЫНЧЕ ВЕТРЕНО И ВОЛНЫ ОТ ПУТИ ТО ВВЕРХ ТО ВНИЗ КАК
МЛАДЕНЦУ ТОГДА В ВИФЛЕЕМЕ ВСЕ БЫЛО ВПРАВДУ ВСЕ В ИХ
ГИБРИДЕ ВИДИТЕ ЛИ БЫЛ ДОВОЛЬНО СТРАННЫМ И КРУГУ
СКВОЗЬ РОЩУ И ГАТЬ И ДОЛГО ПОЛЗ ПОТОМ НУ В СМЫСЛЕ НА СТОЛ


4-GRAMMS

In [22]:
# Define the Markov model
premodel = nltk.probability.ConditionalFreqDist()

words = nltk.word_tokenize(text)

for i in range(len(words)-3):
    key = words[i] + " " + words[i+1] + " " + words[i+2]
    premodel[key][words[i+3]] += 1

In [23]:
model = nltk.ConditionalProbDist(premodel, nltk.MLEProbDist)
len(model)

159539

In [25]:
def generate_poem_quad(model, num_lines, num_syllables, start_condition):
     
    # Создаем список для хранения сгенерированных строк
    lines = []

    # Создаем начальное условие 
    condition = start_condition

    # Создаем список всех слов
    all_words = condition.split()
    
    # Генерируем каждую строку
    for i in range(num_lines):

        if i == 0: #If this start line
            line_words = condition.split()
            line_syllables = count_syllables(line_words[-3]) + count_syllables(line_words[-2]) + count_syllables(line_words[-1]) 
        else:
            line_words = []
            line_syllables = 0

        # Генерируем строку, пока количество слогов не достигнет заданного значения
        while line_syllables < num_syllables:

            # Получаем вероятностное распределение для следующего слова и генерируем его
            word = model[condition].generate()

            # Считаем количество слогов в слове
            syllable_count = count_syllables(word)
            
            # Если количество слогов превышает оставшееся количество для строки, то выбираем новое слово КОСТЫЛЬ
            if line_syllables + syllable_count > num_syllables:
                condition = random.choice(list(model.conditions()))
                all_words = all_words[:-3] + condition.split()
                continue

            # Добавляем слово в строку и увеличиваем количество слогов
            line_words.append(word)
            all_words.append(word)
            line_syllables += syllable_count
                        
            # Обновляем текущее условие для следующей генерации
            condition = all_words[-3] + " " + all_words[-2] + " " + all_words[-1]

       # Добавляем строку в список
        lines.append(' '.join(line_words))
        
    # Возвращаем сгенерированные строки
    return lines

In [27]:
num_lines = 4
num_syllables = 16

# Создаем начальное условие из случайных слов в модели
start_condition = random.choice(list(model.conditions()))

#Генерируем
poem = generate_poem_quad(model, num_lines, num_syllables, start_condition)

#Выводим
for line in poem:
    print(line)

НЕБЕС ПО ЗАСТЫВШЕЙ ТРАВЕ МИМО ЧЕРНЫХ КУСТОВ ХРАНИТ
ОТВЕТ ИРИНА Я СЛУШАЮ ВЗГЛЯНИ-КА СЮДА НИЧЬЯ
И БОЛЬШАК РАЗВЕЗЛО ХОТЬ БЕРИ ВЕСЛО УРОНИЛ ЕСТЬ ТА
ЧЕРТА ЧТО ПО КРАЙНЕЙ МЕРЕ ОБЪЕДИНЯЕТ Е С И МОЙ


In [29]:
num_lines = 4
num_syllables = 16

# Создаем начальное условие из случайных слов в модели
start_condition = "НЫНЧЕ ВЕТРЕНО И"

#Генерируем
poem = generate_poem_quad(model, num_lines, num_syllables, start_condition)

#Выводим
for line in poem:
    print(line)

НЫНЧЕ ВЕТРЕНО И ВОЛНЫ С ПЕРЕХЛЕСТОМ СКОРО ОСЕНЬ
ВСЕ ИЗМЕНИТСЯ В ОКРУГЕ СМЕНА КРАСОК ЭТИХ ТЕМ МЫСЛЬ
О КАМУФЛЯЖЕ ТАМ НА КРЕСТЕ НЕ ВОЗОПЛЮ ПОЧТО ТОЙ
СВЕЧИ ЧТО ВСЕМИ ИСААКОМ ПРЕЖДЕ ЗВАЛАСЬ И ЗВУК ЛИ
